# Importing Libraries

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import tensorflow as tf
from deep_sort_realtime.deepsort_tracker import DeepSort

In [ ]:
yolo_model = YOLO("yolov8m.pt")
MoBiLSTM_model = tf.keras.models.load_model('') # Path of the file 'model(bonus).keras'
tracker = DeepSort(max_age=100, nn_budget=100, embedder="mobilenet", n_init=3)

IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
CLASS_LABELS = ['Brawl', 'Peace']

# Helper Functions

In [ ]:
def auto_text_size(img_width, base_scale=0.9):  # Increased base_scale for better clarity
    """ Adjust text size dynamically based on image width """
    scale = max(base_scale * (img_width / 640), 0.7)  # Minimum scale increased
    thickness = max(int(scale * 3), 2)  # Increased thickness for visibility
    return scale, thickness

In [ ]:
def detect_people_and_classify(video_path, output_video_path="fight_detection.mp4"):
    cap = cv2.VideoCapture(video_path)
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
    width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), frame_rate, (width, height))

    max_brawl_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = yolo_model(frame)
        boxes = results[0].boxes.xyxy.cpu().numpy()
        scores = results[0].boxes.conf.cpu().numpy()
        class_ids = results[0].boxes.cls.cpu().numpy()

        detections = [
            [[box[0], box[1], box[2], box[3]], score, int(cls)]
            for box, score, cls in zip(boxes, scores, class_ids) if int(cls) == 0
        ]

        track_objects = tracker.update_tracks(detections, frame=frame)

        current_brawl_count = 0

        for track in track_objects:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue

            track_id = track.track_id
            ltrb = track.to_ltrb()
            x1, y1, x2, y2 = map(int, ltrb)

            person_crop = frame[y1:y2, x1:x2]
            if person_crop.size == 0:
                continue

            person_crop = cv2.resize(person_crop, (IMAGE_HEIGHT, IMAGE_WIDTH)) / 255.0
            person_crop = np.expand_dims(person_crop, axis=0)

            pred = MoBiLSTM_model.predict(np.array([person_crop]))
            label_index = np.argmax(pred)
            label = CLASS_LABELS[label_index]

            if label == "Brawl":
                current_brawl_count += 1
                color = (0, 0, 255)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)

                font_scale, thickness = auto_text_size(width)
                cv2.putText(frame, f"ID {track_id}: {label}", (x1, y1 - 15),
                            cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness, cv2.LINE_AA)

        max_brawl_count = max(max_brawl_count, current_brawl_count)

        font_scale, thickness = auto_text_size(width)

        cv2.putText(frame, f"Max Brawl Count: {max_brawl_count}", (0, 0),
                    cv2.FONT_HERSHEY_SIMPLEX, int(font_scale/2), (255, 0, 0), int(thickness/5), cv2.LINE_AA)

        out.write(frame)

    cap.release()
    out.release()
    print(f"Number of People involved in brawl: {max_brawl_count}")
    print(f"Fight detection video saved as {output_video_path}")

# Sample

In [ ]:
video_path = '' # Path of any video file
detect_people_and_classify(video_path)